In [21]:
%load_ext autoreload
from path import Path2D
from interpolator import LinearInterpolator

import numpy as np
import matplotlib.pyplot as plt


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [318]:
%autoreload 2
N = 100
tmax = 30
time = np.linspace(0, tmax, N)
a = 0.3
v0 = -3
x = 0.05*time**3/3 +a*time**2 / 2+ v0 *time
y = v0*time
plt.scatter(x, y)
# plt.scatter(-x, y)

# plt.plot(time, time + np.sin(time * 8*np.pi/tmax))

nodes = np.array([x, y]).T
nodes2 = np.array([np.linspace(-1, 1,50), np.linspace(0.2, -1,50)]).T
print(nodes.shape)
path = Path2D(tmax, nodes, LinearInterpolator)
path2 = Path2D(tmax, nodes2, LinearInterpolator)
print(path.__str__())


SyntaxError: invalid syntax (3096932518.py, line 7)

In [240]:
data = np.array([path.pos(t) for t in np.linspace(0, 1, 150)])
data2 = np.array([path2.pos(t) for t in np.linspace(0, 1, 150)])
vel_data = np.array([path.vel(t) for t in np.linspace(0, 1, 150)])
print(data.shape)
plt.figure()
plt.subplot(2,1,1)
plt.plot(data[:,0], data[:,1])
plt.plot(data2[:,0], data2[:,1])
plt.subplot(2,1,2)
plt.plot(vel_data[:-5,0], 'r')
plt.plot(vel_data[:-5,1], 'g')
plt.plot(np.sqrt(vel_data[:-5,0]**2 + vel_data[:-5,1]**2), 'b')

(150, 2)


In [315]:
from radar import RadarGenerator

# probes = [[0.2, 0, 0], [0.6, 0.2, 0], [0.8, 0.2, 0], [0.9, 0.2, 0]]
nprobes = 60
probex = np.linspace(-0.5, 0.5, nprobes)
probey = np.linspace(-0.2, 0.1, nprobes)
probetimes = np.linspace(0, 30, nprobes)
probes = np.array([probetimes, probex, probey]).T

radarGen = RadarGenerator([path], 0.5, 1)
measurements = radarGen.make_measurement_series(probes)

msdata = np.array([[ms.x, ms.y] for ms in measurements])
plt.figure()
# plt.subplot(2,1,1)
plt.scatter(msdata[:,0], msdata[:,1])
# plt.plot(probex, probey)
# plt.subplot(2,1,2)
# plt.scatter(probetimes,[ms.vr for ms in measurements])

In [317]:
from filters.kalman import KalmanFilter
import copy

%matplotlib qt

kf = KalmanFilter(np.array([0, 0, 0, 0, 0, 0]), 0.5)

dt = 0.2
T = 28
steps = int(T / dt)

P_k = []
x_k = []
msrmts = copy.deepcopy(measurements)
for i in range(steps):
    P_k.append(kf.cov)
    x_k.append(kf.prediction)

    kf.predict(dt)
    if i*dt > msrmts[0].t:
        kf.update(np.array([msrmts[0].x, msrmts[0].y]), 2, 5)
        msrmts.pop(0)
        # print('updated')

msdata = np.array([[ms.x, ms.y] for ms in measurements])
x = np.array(x_k)[:,0]
y = np.array(x_k)[:,3]
plt.scatter(x,y, color='red')
plt.scatter(msdata[:,0], msdata[:,1], color='blue')
    